# Modules

## Structures

### Defining structures

All OCaml programs are organised into *modules*. The simplest form of module is a *structure*. You can think of structures as collections of definitions. Structures can be created using the `module` and `struct` keywords:

In [1]:
module M = struct
  type t = T
  let x = T
end

### Accessing structure components

The components of a module can be accessed using the `.` operator:

In [2]:
let y = M.x

Note that the `.` operator works for types as well as values: the `y` variable defined above has type `M.t`.

### Files as structures

In OCaml every source file defines a structure. For example, a file called `foo.ml` would be treated as the definition of a module called `Foo`. We have already used such modules in earlier examples: for instance the `List.map` function of the standard library is defined in a file called [`list.ml`](https://github.com/ocaml/ocaml/blob/trunk/stdlib/list.ml#L90).

## Signatures and abstraction

Just as all values in OCaml have a type, all modules have a *module type*. As you can see from the output, the module `M` defined above has the module type:
````
sig
  type t = T
  val x : t
end
````
This means that it contains a variant type `t` with a single `T` constructor, and a value `x` of type `t`. The module types of structures -- like the one above -- are often called *signatures*.

### Signature ascription

Whilst OCaml will infer the module type of a structure from its definition, you can also ascribe it a more restricted signature. This allows us to hide some of the details of the structure:

In [3]:
module IntSet : sig
  type t
  val empty: t
  val mem: int -> t -> bool
  val add: int -> t -> t
end = struct
  type t = int list

  let empty = []

  let mem i s =
    let is_i j = (i = j) in
      List.exists is_i s

  let add i s =
    if mem i s then s
    else i :: s
end

Here we create an `IntSet` module with a type `t` representing sets of integers.

In [4]:
let s = IntSet.add 6 (IntSet.add 5 IntSet.empty)

let b = IntSet.mem 6 s

By not including the defintion of `t` in the signature, we hide the implementation of `IntSet`. This means that users of our set type cannot depend on the fact we have implemented it using lists.

In [5]:
let r = 4 :: s

Later we can switch to a more efficient implementation using trees safe in the knowledge that this will not break existing code using `IntSet`.

Types with hiddent definitions -- like `t` above -- are called abstract types. OCaml's support for abstraction is one of its most important and powerful features.

### Signatures for files

To add a signature to the module represented by a file we add an interface file. For example, if a file called `foo.ml` defines a structure called `Foo` then `foo.mli` defines the signature of `Foo`. Corresponding to the [`list.ml`](https://github.com/ocaml/ocaml/blob/trunk/stdlib/list.ml) in the OCaml standar library, we have [`list.mli`](https://github.com/ocaml/ocaml/blob/trunk/stdlib/list.mli) which describes the signature of the list interface.

## Functors

In OCaml, functors are module level functions that take modules as arguments and return other modules as results. We had earlier seen an identity function:


In [6]:
let id (x : int) = x

that takes some integer and returns the same. We can define a similar identity functor at the module level. First, let us define a module type which only containts the type alias for `int` type.

In [7]:
module type Int = sig
  type t = int
end

Now we can define the functor as follows:

In [8]:
module Id (X: Int) : Int = X

The type says that `Id` is a functor which takes a module of type `Int` and returns another module of the same module type `Int`. We can apply the functor to a module that satisfies this signature as follows:

In [9]:
module S = Id(struct type t = int let v = 10 end)

More usefully, with the help of functors we can define a set data structure over arbitrary data type:

In [10]:
module Set (Content: sig type t end): sig
  type t
  val empty: t
  val mem: Content.t -> t -> bool
  val add: Content.t -> t -> t
end = struct
  type t = Content.t list

  let empty : t = []

  let mem i s =
    let is_i j = (i = j) in
      List.exists is_i s

  let add i s =
    if mem i s then s
    else i :: s
end

We can use this to create a set of integers:

In [11]:
module IntSet = Set(struct type t = int end)

In [12]:
let is = IntSet.add 1 (IntSet.add 2 (IntSet.empty))

or floats:

In [13]:
module FloatSet = Set(struct type t = float end)

In [14]:
let fs = FloatSet.add 0.1 (FloatSet.add 0.2 (FloatSet.empty))

val fs : FloatSet.t = <abstr>


or set of set of integers:

In [15]:
module IntSetSet = Set(IntSet)

module IntSetSet :
  sig
    type t = Set(IntSet).t
    val empty : t
    val mem : IntSet.t -> t -> bool
    val add : IntSet.t -> t -> t
  end


In [16]:
let iis = IntSetSet.add is IntSetSet.empty

val iis : IntSetSet.t = <abstr>
